This notebook computes on the freely available GTEX version 8 files on Amazon AWS using the Seven Bridges Cancer Genomics Cloud WES service. 

In this case the DRS ids of the GTEX files are available via a client which acccess a local manifest file. The manifest file is obtained from Anvil as follows.
https://anvilproject.org/learn/reference/gtex-v8-free-egress-instructions

Rather than use the Gen3 Downloader described in the egress instructions, this example uses the Anvil DRS service to obtain URLs which can be passed to the Seven Bridges CGC WES Service.

The following step demonstrates a query run against that file. 

In [1]:
from fasp.runner import FASPRunner

# The implementations we're using
from fasp.workflow import DNAStackWESClient
from fasp.loc import anvilDRSClient
from fasp.search import Gen3ManifestClient

faspRunner = FASPRunner(program='GTEXExample')

# Step 1 - Discovery
# query for relevant DRS objects
searchClient = Gen3ManifestClient('../fasp/data/gtex/gtex-cram-manifest.json')
res = searchClient.runQuery(3)


In [2]:
drsClient = anvilDRSClient('~/.keys/anvil_credentials.json', access_id='gs')
drsClient.getObject(res[2][1])

{'access_methods': [{'access_id': 'gs',
   'access_url': {'url': 'gs://fc-secure-ff8156a3-ddf3-42e4-9211-0fd89da62108/GTEx_Analysis_2017-06-05_v8_WGS_CRAM_files/GTEX-1B98T-0004-SM-7J38T.cram'},
   'region': '',
   'type': 'gs'},
  {'access_id': 's3',
   'access_url': {'url': 's3://AnVIL/GTEx_Analysis_2017-06-05_v8_WGS_CRAM_files/GTEX-1B98T-0004-SM-7J38T.cram'},
   'region': '',
   'type': 's3'}],
 'aliases': [],
 'checksums': [{'checksum': 'cfd2f4cde4aa3e0cf9f726f0c1255fcd',
   'type': 'md5'}],
 'contents': [],
 'created_time': '2020-07-08T18:53:27.860156',
 'description': None,
 'form': 'object',
 'id': 'dg.ANV0/ed9ac9ae-02da-4e97-93da-ad86aa77d227',
 'mime_type': 'application/json',
 'name': 'GTEX-1B98T-0004-SM-7J38T.cram',
 'self_uri': 'drs://gen3.theanvil.io/dg.ANV0/ed9ac9ae-02da-4e97-93da-ad86aa77d227',
 'size': 38379493977,
 'updated_time': '2020-07-08T18:53:27.860163',
 'version': '64acd5a6'}

In [3]:
url = drsClient.getAccessURL(res[2][1])

In [4]:
settings = faspRunner.settings
wesClient = DNAStackWESClient('~/.keys/dnastack_wes_credentials.json')

faspRunner.configure(searchClient, drsClient, wesClient)

runList = faspRunner.runQuery(3, 'Anvil GTEX DRS to DNAStack WES')

Running query
3
subject=GTEX-1GTWX-0001-SM-7J3A5.cram, drsID=dg.ANV0/76bb893d-12da-41ca-8828-ff89551d3e15
workflow submitted, run:2f73492e-d109-4e1a-b2be-9950deae9890
subject=GTEX-14PQA-0003-SM-7DLH4.cram, drsID=dg.ANV0/66352de8-4b50-4cae-881d-b76d03df5ac8
workflow submitted, run:6c63ef08-e61b-43a8-ab07-7975a98bdbb8
subject=GTEX-1B98T-0004-SM-7J38T.cram, drsID=dg.ANV0/ed9ac9ae-02da-4e97-93da-ad86aa77d227
workflow submitted, run:1d764abd-5c8f-4c89-ad2b-26a43b2025f1


In [7]:
for run in runList:
    print(run['run_id'], wesClient.getTaskStatus(run['run_id']))

2f73492e-d109-4e1a-b2be-9950deae9890 RUNNING
6c63ef08-e61b-43a8-ab07-7975a98bdbb8 RUNNING
1d764abd-5c8f-4c89-ad2b-26a43b2025f1 RUNNING


In [8]:
print (runList)

[{'subject': 'GTEX-1GTWX-0001-SM-7J3A5.cram', 'run_id': '2f73492e-d109-4e1a-b2be-9950deae9890'}, {'subject': 'GTEX-14PQA-0003-SM-7DLH4.cram', 'run_id': '6c63ef08-e61b-43a8-ab07-7975a98bdbb8'}, {'subject': 'GTEX-1B98T-0004-SM-7J38T.cram', 'run_id': '1d764abd-5c8f-4c89-ad2b-26a43b2025f1'}]
